In [10]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date, StringType as Str

In [11]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark = create_spark_session()

In [12]:
song_input_data = "data/more/song_data/A/A/A/TRAAABD128F429CF47.json"
    #or data/more/song-data/*.json"
    
    # read song data file
songSchema = R([
    Fld("num_songs",Int()),
    Fld("artist_id",Str()),
    Fld("artist_latitude",Dbl()),
    Fld("artist_longitude",Dbl()),
    Fld("artist_location",Str()),
    Fld("artist_name",Str()),
    Fld("song_id",Str()),
    Fld("title",Str()),
    Fld("duration",Dbl()),
    Fld("year",Int())
])
df = spark.read.format("json").load(song_input_data, schema=songSchema)

In [13]:
song_input_data

'data/more/song_data/A/A/A/TRAAABD128F429CF47.json'

In [5]:
print(df.count())
df.printSchema()

1
root
 |-- num_songs: integer (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)



In [6]:
df.show()

+---------+------------------+---------------+----------------+---------------+------------+------------------+---------+---------+----+
|num_songs|         artist_id|artist_latitude|artist_longitude|artist_location| artist_name|           song_id|    title| duration|year|
+---------+------------------+---------------+----------------+---------------+------------+------------------+---------+---------+----+
|        1|ARMJAGH1187FB546F3|       35.14968|       -90.04892|    Memphis, TN|The Box Tops|SOCIWDW12A8C13D406|Soul Deep|148.03546|1969|
+---------+------------------+---------------+----------------+---------------+------------+------------------+---------+---------+----+



In [7]:
year = "year"
artist = "artist_id"
dfYearArtist = df.select(year, artist)
dfYearArtist.limit(5).toPandas()

year           artist_id
0  1969  ARMJAGH1187FB546F3

In [14]:
# define output bucket und AWS S3
#output = "s3a://sparkifyphil.s3.us-east-2.amazonaws.com/output/song_data/"
output="partitioned2"
# create schema-on-read for table "songs"
df.createOrReplaceTempView("songs")
songs_table = spark.sql("""
    SELECT     
        DISTINCT song_id,
        title,
        artist_id,
        year,
        duration
    FROM songs
""")


# write songs table to a parquet file
songs_table.write.partitionBy("year", "artist_id").parquet(output + "-song_table")

In [16]:
artists_table = spark.sql("""
    SELECT  
        DISTINCT artist_id,
        artist_name AS name, 
        artist_location AS location, 
        artist_latitude AS lattitude,
        artist_longitude AS longitude
    FROM songs  

""")

artists_table.write.parquet(output+"-artist_table")